In [1]:
import pandas as pd
from pytube import YouTube
from os import mkdir, remove
from os.path import join, splitext, basename
import subprocess
import glob
from tqdm import tqdm

In [2]:
train_csv_path = "../resources/downloaded/kinetics700_2020/train.csv"
output_dir = "../resources/downloaded/kinetics700_2020/videos"

In [10]:
mkdir(output_dir)

FileExistsError: [Errno 17] File exists: '../resources/downloaded/kinetics700_2020/videos'

In [3]:
df_train = pd.read_csv(train_csv_path, sep=",", header=0)

In [4]:
failed_youtube_id = set()

In [5]:
already_downloaded_videos = set(splitext(basename(f))[0] for f in glob.glob(join(output_dir, f"*.mp4")))

In [ ]:
tqdm_bar = tqdm(list(df_train.iterrows())[144061+210000:])
skipped = 0
failed = 0
done = 0

for i, row in tqdm_bar:
    youtube_id = row["youtube_id"]
    time_start = row["time_start"]
    time_end = row["time_end"]

    if youtube_id in already_downloaded_videos or youtube_id in failed_youtube_id:
        skipped += 1
        continue

    label = row["label"]
    try:
        (
            YouTube(f"https://youtube.com/watch?v={youtube_id}")
            .streams
            .filter(res="360p", fps=30, file_extension="mp4")
            .first()
            .download(output_dir, filename=f"tmp_{youtube_id}.mp4")
        )

        subprocess.run(
            [
                "ffmpeg",
                "-y",
                "-ss",
                str(time_start),
                "-i",
                join(output_dir, f"tmp_{youtube_id}.mp4"),
                "-t",
                "10",
                "-c",
                "copy",
                join(output_dir, f"{youtube_id}.mp4"),
                "-hide_banner",
                "-loglevel",
                "error",
            ]
        )
        remove(join(output_dir, f"tmp_{youtube_id}.mp4"))

        done += 1
    except Exception:
        failed += 1
        failed_youtube_id.add(youtube_id)

    tqdm_bar.set_description(f"Skipped {skipped}, failed {failed}, done {done}")

Skipped 57, failed 12522, done 156:   7%|▋         | 12735/178845 [2:34:18<32:47:12,  1.41it/s] 